# 🐑 Understanding Object Copying in Python: Shallow vs. Deep Copy

**Welcome!** When working with objects in Python, especially mutable ones like lists and dictionaries, simply assigning one variable to another (`b = a`) doesn't create a true copy; it creates another reference to the *same* object. This notebook explores the crucial difference between assignment, shallow copies, and deep copies, and why understanding this is vital for avoiding unintended side effects.

**Target Audience:** Python developers who need to duplicate objects, particularly mutable or nested ones, without modifying the original.

**Learning Objectives:**
*   Understand how Python assignment creates references, not copies.
*   Create shallow copies using `copy.copy()`, slicing, type constructors, and `.copy()` methods.
*   Recognize the limitations of shallow copies with nested mutable objects.
*   Create fully independent deep copies using `copy.deepcopy()`.
*   Understand how copying works with custom objects.
*   Learn how to customize copying behavior using `__copy__` and `__deepcopy__`.
*   Identify best practices and pitfalls related to object copying.

## 1. Introduction: The Need for Copies

Imagine you have a list representing project tasks, and you want to create a backup or a modified version without altering the original list. If you just assign it to a new variable, both variables point to the *exact same list* in memory. Modifying one will modify the other.

**Analogy: The Shared Document vs. Photocopies**

*   **Assignment (`b = a`):** Like giving someone else a link to the *same* shared online document. If they edit it, your view of the document changes too.
*   **Shallow Copy:** Like making a photocopy of the *first page* of a multi-page, stapled report. You have a separate first page, but if the report contains references (staples) to other pages (nested objects), your copy still points to the *original* subsequent pages. Changing those original subsequent pages will be reflected when you look through your "copy".
*   **Deep Copy:** Like making a complete, independent photocopy of the *entire* report, including all nested pages. Your copy is fully standalone; changes to the original report don't affect your copy, and vice-versa.

Understanding which type of copy you need is crucial for correct program behavior, especially when dealing with mutable data.

## 2. Assignment: Sharing References

The assignment operator (`=`) in Python binds a name (variable) to an object. Assigning one variable to another simply makes both names point to the same object in memory.

In [1]:
print("--- Assignment (Reference Sharing) ---")
list_a = [1, 2, [30, 40]]
list_b = list_a # list_b now refers to the SAME list object as list_a

print(f"list_a: {list_a}")
print(f"list_b: {list_b}")
print(f"ID of list_a: {id(list_a)}")
print(f"ID of list_b: {id(list_b)}") # Same ID confirms they are the same object
print(f"Are list_a and list_b the same object? {list_a is list_b}")

# Modify list_b (which is the same object as list_a)
list_b.append(50)
list_b[0] = 100
list_b[2].append(99) # Modify the nested list

print("\n--- After modifying list_b --- ")
print(f"list_a: {list_a}") # list_a is also changed!
print(f"list_b: {list_b}")

--- Assignment (Reference Sharing) ---
list_a: [1, 2, [30, 40]]
list_b: [1, 2, [30, 40]]
ID of list_a: 135715183499264
ID of list_b: 135715183499264
Are list_a and list_b the same object? True

--- After modifying list_b --- 
list_a: [100, 2, [30, 40, 99], 50]
list_b: [100, 2, [30, 40, 99], 50]


**Conclusion:** Assignment never creates a copy of the object itself, only copies the reference.

## 3. Shallow Copy: Copying the Top Level

A shallow copy creates a **new compound object** (like a new list or dict) but then populates it with **references** to the objects found in the original.

*   The top-level container is duplicated.
*   The elements *inside* the container are **not** recursively copied; the new container holds references to the *same* elements as the original.

**How to Create Shallow Copies:**
1.  `copy.copy(original)`: The general function from the `copy` module.
2.  Slicing (`original[:]`): For sequence types like lists and tuples.
3.  Type Constructor (`list(original)`, `dict(original)`, etc.).
4.  `.copy()` Method: Available on some mutable types like `list`, `dict`, `set`.

### 3.1 Shallow Copy with Immutable Elements

In [2]:
import copy

print("--- Shallow Copy (Immutable Elements) ---")
list_a = [1, 2, 3, 4]

# Create shallow copies using different methods
list_b_copy = copy.copy(list_a)
list_c_slice = list_a[:]
list_d_constructor = list(list_a)
list_e_method = list_a.copy()

print(f"Original list_a: {list_a}, ID: {id(list_a)}")
print(f"copy.copy():     {list_b_copy}, ID: {id(list_b_copy)}") # New list object
print(f"Slicing:         {list_c_slice}, ID: {id(list_c_slice)}") # New list object
print(f"list():          {list_d_constructor}, ID: {id(list_d_constructor)}") # New list object
print(f".copy():         {list_e_method}, ID: {id(list_e_method)}") # New list object

# Modify the shallow copy (list_b_copy)
list_b_copy[0] = 100

print("\n--- After modifying shallow copy (list_b_copy) --- ")
print(f"Original list_a: {list_a}") # Unchanged
print(f"Shallow copy:    {list_b_copy}") # Changed

# The inner elements (integers) are immutable. Even though list_a[1] and list_b_copy[1]
# might refer to the same integer object '2' initially, changing list_b_copy[0] 
# makes it point to a *new* integer object '100'. 
print(f"ID of list_a[1]:    {id(list_a[1])}")
print(f"ID of list_b_copy[1]: {id(list_b_copy[1])}") # Same as list_a[1] (integer 2)
print(f"ID of list_b_copy[0]: {id(list_b_copy[0])}") # Different (integer 100)

--- Shallow Copy (Immutable Elements) ---
Original list_a: [1, 2, 3, 4], ID: 135715183618944
copy.copy():     [1, 2, 3, 4], ID: 135715183658176
Slicing:         [1, 2, 3, 4], ID: 135715183536192
list():          [1, 2, 3, 4], ID: 135715183573376
.copy():         [1, 2, 3, 4], ID: 135715183781056

--- After modifying shallow copy (list_b_copy) --- 
Original list_a: [1, 2, 3, 4]
Shallow copy:    [100, 2, 3, 4]
ID of list_a[1]:    135715252749296
ID of list_b_copy[1]: 135715252749296
ID of list_b_copy[0]: 135715252752432


### 3.2 Shallow Copy with Nested Mutable Elements (The Pitfall!)

In [3]:
import copy

print("\n--- Shallow Copy (Nested Mutable Elements) --- ")
list_a = [1, 2, [30, 40]] # Contains a mutable list inside
list_b = list_a.copy() # Create a shallow copy

print(f"Original list_a: {list_a}, ID: {id(list_a)}")
print(f"Shallow copy b:  {list_b}, ID: {id(list_b)}") # Different list object

# Check IDs of elements
print(f"ID of list_a[0]: {id(list_a[0])}")
print(f"ID of list_b[0]: {id(list_b[0])}") # Same integer 1
print(f"ID of list_a[2]: {id(list_a[2])}") # ID of the inner list
print(f"ID of list_b[2]: {id(list_b[2])}") # *** SAME ID as list_a[2] *** 
print(f"Are inner lists the same object? {list_a[2] is list_b[2]}")

# --- Modify elements --- 
print("\n--- Modifying elements --- ")
# Modify top-level element of the copy
list_b[0] = 100 
print(f"Modified top level -> list_a: {list_a}") # Unchanged
print(f"Modified top level -> list_b: {list_b}") # Changed

# Modify element INSIDE the nested list via the shallow copy (list_b)
list_b[2].append(99)
print("\n--- After modifying nested list via shallow copy --- ")
print(f"Original list_a: {list_a}") # !!! Also changed !!!
print(f"Shallow copy b:  {list_b}")

# Modify the nested list via the original (list_a)
list_a[2].append(88)
print("\n--- After modifying nested list via original --- ")
print(f"Original list_a: {list_a}") 
print(f"Shallow copy b:  {list_b}") # !!! Also changed !!!


--- Shallow Copy (Nested Mutable Elements) --- 
Original list_a: [1, 2, [30, 40]], ID: 135715183613696
Shallow copy b:  [1, 2, [30, 40]], ID: 135715183778368
ID of list_a[0]: 135715252749264
ID of list_b[0]: 135715252749264
ID of list_a[2]: 135715183610560
ID of list_b[2]: 135715183610560
Are inner lists the same object? True

--- Modifying elements --- 
Modified top level -> list_a: [1, 2, [30, 40]]
Modified top level -> list_b: [100, 2, [30, 40]]

--- After modifying nested list via shallow copy --- 
Original list_a: [1, 2, [30, 40, 99]]
Shallow copy b:  [100, 2, [30, 40, 99]]

--- After modifying nested list via original --- 
Original list_a: [1, 2, [30, 40, 99, 88]]
Shallow copy b:  [100, 2, [30, 40, 99, 88]]


**Conclusion:** Shallow copies only duplicate the outer container. Nested mutable objects are still shared between the original and the shallow copy. Modifying nested mutable objects through one variable will affect the other.

## 4. Deep Copy: Creating Fully Independent Clones

A deep copy creates a **new compound object** and then, **recursively**, inserts *copies* of the objects found in the original.

*   Both the top-level container and all nested objects (including mutable ones) are duplicated.
*   The original and the deep copy are fully independent.

**How to Create Deep Copies:**
1.  `copy.deepcopy(original)`: The primary function for deep copying.

In [4]:
import copy

print("--- Deep Copy (Nested Mutable Elements) --- ")
list_a = [1, 2, [30, 40]] # Contains a mutable list inside

# Create a deep copy
list_b = copy.deepcopy(list_a) 

print(f"Original list_a: {list_a}, ID: {id(list_a)}")
print(f"Deep copy b:     {list_b}, ID: {id(list_b)}") # Different list object

# Check IDs of elements
print(f"ID of list_a[0]: {id(list_a[0])}")
print(f"ID of list_b[0]: {id(list_b[0])}") # Same integer 1 (immutables might be reused)
print(f"ID of list_a[2]: {id(list_a[2])}") # ID of the inner list in a
print(f"ID of list_b[2]: {id(list_b[2])}") # *** DIFFERENT ID from list_a[2] ***
print(f"Are inner lists the same object? {list_a[2] is list_b[2]}") # False!

# --- Modify elements --- 
print("\n--- Modifying elements --- ")
# Modify top-level element of the deep copy
list_b[0] = 100 
print(f"Modified top level -> list_a: {list_a}") # Unchanged
print(f"Modified top level -> list_b: {list_b}") # Changed

# Modify element INSIDE the nested list via the deep copy (list_b)
list_b[2].append(99)
print("\n--- After modifying nested list via deep copy --- ")
print(f"Original list_a: {list_a}") # *** Unchanged ***
print(f"Deep copy b:     {list_b}")

# Modify the nested list via the original (list_a)
list_a[2].append(88)
print("\n--- After modifying nested list via original --- ")
print(f"Original list_a: {list_a}") 
print(f"Deep copy b:     {list_b}") # *** Unchanged ***

--- Deep Copy (Nested Mutable Elements) --- 
Original list_a: [1, 2, [30, 40]], ID: 135715183779200
Deep copy b:     [1, 2, [30, 40]], ID: 135715183774592
ID of list_a[0]: 135715252749264
ID of list_b[0]: 135715252749264
ID of list_a[2]: 135715183777728
ID of list_b[2]: 135715183775424
Are inner lists the same object? False

--- Modifying elements --- 
Modified top level -> list_a: [1, 2, [30, 40]]
Modified top level -> list_b: [100, 2, [30, 40]]

--- After modifying nested list via deep copy --- 
Original list_a: [1, 2, [30, 40]]
Deep copy b:     [100, 2, [30, 40, 99]]

--- After modifying nested list via original --- 
Original list_a: [1, 2, [30, 40, 88]]
Deep copy b:     [100, 2, [30, 40, 99]]


**Conclusion:** Deep copies create fully independent duplicates, including all nested structures. This is usually what you want when you need to modify a copy without affecting the original, especially with complex, nested data.

## 5. Copying Custom Objects

The `copy` module also works with instances of your custom classes.

*   `copy.copy(instance)`: Creates a new instance of the class and then performs a *shallow* copy of the instance's attributes (usually by copying the instance's `__dict__`). References to nested mutable objects within the instance's attributes will be shared.
*   `copy.deepcopy(instance)`: Creates a new instance and then *recursively* performs a deep copy of the instance's attributes. Nested mutable objects are also duplicated.

In [5]:
import copy

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

class Rectangle:
    def __init__(self, top_left: Point, bottom_right: Point):
        self.top_left = top_left         # Attribute is a custom object
        self.bottom_right = bottom_right # Attribute is a custom object
        self.attributes = ['shape', {'color': 'red'}] # Attribute with nested mutable list/dict
        
    def __repr__(self):
        return f"Rectangle({self.top_left!r}, {self.bottom_right!r}, {self.attributes!r})"

# Create original objects
p1 = Point(10, 20)
p2 = Point(100, 200)
rect_a = Rectangle(p1, p2)

print("--- Custom Object Copying --- ")
print(f"Original rect_a: {rect_a}, ID: {id(rect_a)}")
print(f"  rect_a.top_left ID: {id(rect_a.top_left)}")
print(f"  rect_a.attributes[1] ID: {id(rect_a.attributes[1])}")

# --- Shallow Copy Custom Object --- 
rect_b_shallow = copy.copy(rect_a)
print("\n--- Shallow Copy (rect_b_shallow) --- ")
print(f"Shallow copy b: {rect_b_shallow}, ID: {id(rect_b_shallow)}") # New Rectangle object
print(f"  rect_b.top_left ID: {id(rect_b_shallow.top_left)}") # SAME Point object as rect_a
print(f"  rect_b.attributes[1] ID: {id(rect_b_shallow.attributes[1])}") # SAME dict object as rect_a

# Modify nested Point object via shallow copy
rect_b_shallow.top_left.x = 111
# Modify nested dict object via shallow copy
rect_b_shallow.attributes[1]['color'] = 'blue'

print("\n--- After modifying nested objects via shallow copy --- ")
print(f"Original rect_a: {rect_a}") # Also modified!
print(f"Shallow copy b:  {rect_b_shallow}")

# --- Deep Copy Custom Object --- 
# Reset original
rect_a.top_left.x = 10 
rect_a.attributes[1]['color'] = 'red'

rect_c_deep = copy.deepcopy(rect_a)
print("\n--- Deep Copy (rect_c_deep) --- ")
print(f"Deep copy c: {rect_c_deep}, ID: {id(rect_c_deep)}") # New Rectangle object
print(f"  rect_c.top_left ID: {id(rect_c_deep.top_left)}") # DIFFERENT Point object from rect_a
print(f"  rect_c.attributes[1] ID: {id(rect_c_deep.attributes[1])}") # DIFFERENT dict object from rect_a

# Modify nested Point object via deep copy
rect_c_deep.top_left.x = 222
# Modify nested dict object via deep copy
rect_c_deep.attributes[1]['color'] = 'green'

print("\n--- After modifying nested objects via deep copy --- ")
print(f"Original rect_a: {rect_a}") # Unchanged!
print(f"Deep copy c:     {rect_c_deep}")

--- Custom Object Copying --- 
Original rect_a: Rectangle(Point(10, 20), Point(100, 200), ['shape', {'color': 'red'}]), ID: 135715183546720
  rect_a.top_left ID: 135715183547728
  rect_a.attributes[1] ID: 135715183890048

--- Shallow Copy (rect_b_shallow) --- 
Shallow copy b: Rectangle(Point(10, 20), Point(100, 200), ['shape', {'color': 'red'}]), ID: 135715183642576
  rect_b.top_left ID: 135715183547728
  rect_b.attributes[1] ID: 135715183890048

--- After modifying nested objects via shallow copy --- 
Original rect_a: Rectangle(Point(111, 20), Point(100, 200), ['shape', {'color': 'blue'}])
Shallow copy b:  Rectangle(Point(111, 20), Point(100, 200), ['shape', {'color': 'blue'}])

--- Deep Copy (rect_c_deep) --- 
Deep copy c: Rectangle(Point(10, 20), Point(100, 200), ['shape', {'color': 'red'}]), ID: 135715183642896
  rect_c.top_left ID: 135715183643216
  rect_c.attributes[1] ID: 135715183873408

--- After modifying nested objects via deep copy --- 
Original rect_a: Rectangle(Point(10, 

## 6. Customizing Copy Behavior: `__copy__` and `__deepcopy__`

Sometimes, the default shallow or deep copy behavior isn't quite right for your custom class. You might want to:
*   Exclude certain attributes from being copied (e.g., temporary state, non-pickleable resources like file handles or network connections).
*   Re-initialize certain attributes in the copy (e.g., reset a cache).
*   Implement custom logic for copying complex internal structures.

You can define special methods in your class:
*   `__copy__(self)`: Called by `copy.copy()`. Should return a shallow copy of the instance.
*   `__deepcopy__(self, memo)`: Called by `copy.deepcopy()`. Should return a deep copy of the instance. The `memo` dictionary is used by `deepcopy` to handle circular references and avoid infinite recursion – you should generally pass it along when deep copying attributes.

In [6]:
import copy

class DatabaseConnection:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        self._connection = None # Represents the actual connection (e.g., socket)
        self._cache = {} # Some internal mutable state
        print(f"Initialized DB Connection for {self.connection_string}")

    def connect(self):
        print(f"Simulating connection to {self.connection_string}...")
        self._connection = object() # Simulate an active connection object
        self._cache = {} # Clear cache on connect

    def close(self):
        print(f"Closing connection for {self.connection_string}...")
        self._connection = None
        
    def add_to_cache(self, key, value):
        self._cache[key] = value
        
    def __repr__(self):
        state = 'Connected' if self._connection else 'Disconnected'
        return f"<DB Conn({self.connection_string!r}), State: {state}, Cache Size: {len(self._cache)}>"
        
    # --- Custom Copy Methods --- 
    
    def __copy__(self):
        """Shallow copy: Create new instance but don't copy connection or cache."""
        print(f"*** Custom __copy__ called for {self!r} ***")
        # Create a new instance with the same config
        new_copy = self.__class__(self.connection_string)
        # Note: We explicitly DO NOT copy _connection or _cache 
        # They should be established/populated independently for the new object
        return new_copy
        
    def __deepcopy__(self, memo: dict = {}):
        """Deep copy: Create new instance, deep copy config if mutable, don't copy connection."""
        # memo is crucial to handle circular references and avoid infinite loops
        print(f"*** Custom __deepcopy__ called for {self!r} ***")
        print(f"    Memo dict ID: {id(memo)}")
        
        # Check if this object is already being copied (part of a cycle)
        if id(self) in memo:
            return memo[id(self)]
            
        # Deep copy attributes that should be copied (e.g., config strings)
        # If connection_string was a mutable object, we'd deepcopy it here:
        # new_conn_string = copy.deepcopy(self.connection_string, memo)
        new_conn_string = self.connection_string # Strings are immutable, simple assignment ok
        
        # Create a new instance for the copy
        new_copy = self.__class__(new_conn_string)
        
        # Store the new copy in the memo dict *before* copying attributes
        # to handle potential cycles within attributes.
        memo[id(self)] = new_copy 
        
        # Deep copy other relevant attributes (e.g., maybe cache? design decision)
        # For this example, let's give the copy a fresh cache
        # new_copy._cache = copy.deepcopy(self._cache, memo) # If we wanted to copy cache
        new_copy._cache = {} # Fresh cache for the copy
        
        # DO NOT copy the _connection state - the copy needs its own connection
        new_copy._connection = None
        
        return new_copy

# --- Testing Custom Copying --- 
conn_a = DatabaseConnection("db://user:pass@host1/prod")
conn_a.connect()
conn_a.add_to_cache("key1", "value1")
print(f"Original conn_a: {conn_a}")

# Shallow copy using copy.copy()
conn_b_shallow = copy.copy(conn_a)
print(f"Shallow copy b:  {conn_b_shallow}") # State is reset

# Deep copy using copy.deepcopy()
conn_c_deep = copy.deepcopy(conn_a)
print(f"Deep copy c:     {conn_c_deep}") # State is reset, cache is fresh

# Verify independence
conn_c_deep.connect()
conn_c_deep.add_to_cache("key_c", "value_c")
print("\n--- After modifying deep copy --- ")
print(f"Original conn_a: {conn_a}") # Unchanged
print(f"Deep copy c:     {conn_c_deep}")

Initialized DB Connection for db://user:pass@host1/prod
Simulating connection to db://user:pass@host1/prod...
Original conn_a: <DB Conn('db://user:pass@host1/prod'), State: Connected, Cache Size: 1>
*** Custom __copy__ called for <DB Conn('db://user:pass@host1/prod'), State: Connected, Cache Size: 1> ***
Initialized DB Connection for db://user:pass@host1/prod
Shallow copy b:  <DB Conn('db://user:pass@host1/prod'), State: Disconnected, Cache Size: 0>
*** Custom __deepcopy__ called for <DB Conn('db://user:pass@host1/prod'), State: Connected, Cache Size: 1> ***
    Memo dict ID: 135715183539712
Initialized DB Connection for db://user:pass@host1/prod
Deep copy c:     <DB Conn('db://user:pass@host1/prod'), State: Disconnected, Cache Size: 0>
Simulating connection to db://user:pass@host1/prod...

--- After modifying deep copy --- 
Original conn_a: <DB Conn('db://user:pass@host1/prod'), State: Connected, Cache Size: 1>
Deep copy c:     <DB Conn('db://user:pass@host1/prod'), State: Connected, 

## 7. Best Practices & Considerations

1.  **Default to Deep Copy for Isolation:** If you need a truly independent copy of a complex or nested mutable object to modify without side effects, use `copy.deepcopy()`.
2.  **Use Shallow Copy for Performance (When Safe):** If you are only copying flat structures or structures with only immutable elements, or if you *intentionally* want shared references to nested objects, a shallow copy (`copy.copy()`, `.copy()`, slicing) is faster and uses less memory.
3.  **Be Aware of Nested Mutability:** The most common source of bugs is misunderstanding how shallow copies handle nested mutable objects.
4.  **Custom Objects:** Understand the default copy behavior for your classes and implement `__copy__` / `__deepcopy__` if you need to customize it (especially to handle resources like connections or to avoid copying large caches unnecessarily).
5.  **Performance Cost:** `deepcopy` can be significantly slower than `copy` because it has to traverse and duplicate the entire object graph. Profile if performance is critical.
6.  **Circular References:** `deepcopy` handles circular references correctly by using the `memo` dictionary, but be aware they exist.
7.  **Non-Pickleable Objects:** `deepcopy` might fail if the object graph contains objects that cannot be pickled (like generators, file handles in some states, certain closures).

## 8. Pitfalls and Common Interview Questions

**Common Pitfalls:**
*   **Using Assignment (`=`) Expecting a Copy:** The most fundamental error.
*   **Using Shallow Copy on Nested Structures:** Modifying nested mutable objects in the copy unintentionally affects the original.
*   **Performance Impact of `deepcopy`:** Using `deepcopy` unnecessarily on large or simple structures where a shallow copy would suffice.
*   **Forgetting `memo` in `__deepcopy__`:** Can lead to infinite recursion if the object graph has cycles.
*   **Trying to Copy Uncopyable Objects:** Some objects (like generators, file handles) inherently cannot or should not be copied meaningfully.

**Common Interview Questions:**

1.  What is the difference between assignment (`b = a`), shallow copy (`b = copy.copy(a)`), and deep copy (`b = copy.deepcopy(a)`) in Python?
2.  Give an example where modifying a shallow copy *would* affect the original object.
3.  When would you typically use a shallow copy versus a deep copy?
4.  How does `copy.deepcopy()` handle nested objects?
5.  How does `copy.deepcopy()` handle circular references?
6.  Can you copy instances of custom classes? How does it work by default?
7.  How can you customize the copying behavior of your own class? (Mention `__copy__` and `__deepcopy__`).

## 9. Challenge: Configuration Management

**Goal:** Simulate modifying different versions of a configuration dictionary safely using copying.

**Tasks:**

1.  **Create Base Configuration:** Define a dictionary `base_config` representing application settings. Include nested structures, like a list of enabled features and a dictionary for database settings.
   ```python
   base_config = {
       'debug_mode': False,
       'log_level': 'INFO',
       'features': ['auth', 'logging'],
       'database': {
           'host': 'db.prod.com',
           'port': 5432,
           'retry_attempts': 3
       }
   }
   ```
2.  **Create Development Config:**
    *   Create a **copy** of `base_config` suitable for development (call it `dev_config`). Choose the appropriate copy method (`copy` or `deepcopy`) based on the modifications you'll make.
    *   Modify `dev_config`: Set `debug_mode` to `True`, change `log_level` to `'DEBUG'`, add `'beta_feature'` to the `features` list, and change the database `host` to `'localhost'`.
    *   Print both `base_config` and `dev_config`. Verify that `base_config` remains unchanged.
3.  **Create Testing Config:**
    *   Create another **copy** of `base_config` suitable for testing (call it `test_config`). Again, choose the appropriate copy method.
    *   Modify `test_config`: Change the database `retry_attempts` to `1`, and remove `'auth'` from the `features` list.
    *   Print both `base_config` and `test_config`. Verify `base_config` is still unchanged.
4.  **Explain Your Choices:** Add markdown cells explaining why you chose shallow or deep copy for creating `dev_config` and `test_config`.

In [7]:
# --- Solution Space for Challenge ---
import copy
import json # For pretty printing dicts

# 1. Create Base Configuration
base_config = {
    'debug_mode': False,
    'log_level': 'INFO',
    'features': ['auth', 'logging'], # Mutable list
    'database': { # Mutable dict
        'host': 'db.prod.com',
        'port': 5432,
        'retry_attempts': 3
    }
}

print("--- Base Configuration ---")
print(json.dumps(base_config, indent=2))

# --- Explanation for Development Config Copy --- 
# We need to modify nested structures ('features' list and 'database' dict) 
# independently of the base configuration. A shallow copy would share these 
# nested objects. Therefore, a DEEP COPY is required to ensure full isolation.

# 2. Create Development Config
dev_config = copy.deepcopy(base_config)

# Modify dev_config
dev_config['debug_mode'] = True
dev_config['log_level'] = 'DEBUG'
dev_config['features'].append('beta_feature') # Modify nested list
dev_config['database']['host'] = 'localhost' # Modify nested dict

print("\n--- Development Configuration (after deepcopy & modify) ---")
print(json.dumps(dev_config, indent=2))

print("\n--- Base Configuration (after dev modify - should be unchanged) ---")
print(json.dumps(base_config, indent=2))

# --- Explanation for Testing Config Copy --- 
# Similar to the development config, we need to modify nested structures 
# ('features' list and 'database' dict - changing retry_attempts) without 
# affecting the base configuration. A DEEP COPY is necessary.

# 3. Create Testing Config
test_config = copy.deepcopy(base_config)

# Modify test_config
test_config['database']['retry_attempts'] = 1 # Modify nested dict
if 'auth' in test_config['features']:
     test_config['features'].remove('auth') # Modify nested list

print("\n--- Testing Configuration (after deepcopy & modify) ---")
print(json.dumps(test_config, indent=2))

print("\n--- Base Configuration (after test modify - should be unchanged) ---")
print(json.dumps(base_config, indent=2))

--- Base Configuration ---
{
  "debug_mode": false,
  "log_level": "INFO",
  "features": [
    "auth",
    "logging"
  ],
  "database": {
    "host": "db.prod.com",
    "port": 5432,
    "retry_attempts": 3
  }
}

--- Development Configuration (after deepcopy & modify) ---
{
  "debug_mode": true,
  "log_level": "DEBUG",
  "features": [
    "auth",
    "logging",
    "beta_feature"
  ],
  "database": {
    "host": "localhost",
    "port": 5432,
    "retry_attempts": 3
  }
}

--- Base Configuration (after dev modify - should be unchanged) ---
{
  "debug_mode": false,
  "log_level": "INFO",
  "features": [
    "auth",
    "logging"
  ],
  "database": {
    "host": "db.prod.com",
    "port": 5432,
    "retry_attempts": 3
  }
}

--- Testing Configuration (after deepcopy & modify) ---
{
  "debug_mode": false,
  "log_level": "INFO",
  "features": [
    "logging"
  ],
  "database": {
    "host": "db.prod.com",
    "port": 5432,
    "retry_attempts": 1
  }
}

--- Base Configuration (after test 

## 10. Conclusion

Understanding the difference between assignment (reference sharing), shallow copies, and deep copies is fundamental in Python, especially when working with mutable data structures like lists and dictionaries, or complex custom objects.

*   Use **assignment** when you want multiple variables to refer to the exact same object.
*   Use **shallow copy** (`copy.copy`, `.copy()`, slicing, constructors) when you need a new top-level container but are okay with (or desire) sharing references to nested objects. It's faster but requires caution with nested mutability.
*   Use **deep copy** (`copy.deepcopy`) when you need a completely independent clone of an object, including all its nested structures, to avoid unintended side effects when modifying the copy.

Choosing the correct copying mechanism prevents subtle bugs and ensures your program behaves as expected when managing object state.